In [1]:
import numpy as np
import sys, os, imp
import cv2
import transformations
import features
import traceback
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage, spatial

from PIL import Image

import pdb
# Saving and loading cv2 points
def pickle_cv2(arr):
    index = []
    for point in arr:
        temp = (point.pt, point.size, point.angle, point.response, point.octave, point.class_id)
        index.append(temp)
    return np.array(index)

def unpickle_cv2(arr):
    index = []
    for point in arr:
        temp = cv2.KeyPoint(x=point[0][0],y=point[0][1], size=point[1], angle=point[2], response=point[3], octave=point[4], class_id=point[5])
        index.append(temp)
    return np.array(index)

# Functions for testing elementwise correctness
def compare_array(arr1, arr2):
    return np.allclose(arr1,arr2,rtol=1e-3,atol=1e-3)

def compare_cv2_points(pnt1, pnt2):
    if not np.isclose(pnt1.pt[0],pnt2.pt[0],rtol=1e-3,atol=1e-5): return False
    if not np.isclose(pnt1.pt[1],pnt2.pt[1],rtol=1e-3,atol=1e-5): return False
    if not np.isclose(pnt1.angle,pnt2.angle,rtol=1e-3,atol=1e-5): return False
    if not np.isclose(pnt1.response,pnt2.response,rtol=1e-3,atol=1e-5): return False
    return True

loaded = np.load('resources/arrays.npz', allow_pickle=True)
d = unpickle_cv2(loaded['d_proc'])

def try_this(todo, run, truth, compare, *args, **kargs):
    '''
    Run a function, test the output with compare, and print and error if it doesn't work
    @arg todo (int or str): The Todo number
    @arg run (func): The function to run
    @arg truth (any): The correct output of the function 
    @arg compare (func->bool): Compares the output of the `run` function to truth and provides a boolean if correct
    @arg *args (any): Any arguments that should be passed to `run`
    @arg **kargs (any): Any kargs that should be passed to compare

    @return (int): The amount of things that failed
    '''
    print('Starting test for TODO {}'.format(todo))
    failed = 0
    try:
        output = run(*args)
    except Exception as e:
        traceback.print_exc()
        print("TODO {} threw an exception, see exception above".format(todo))
        return
    if type(output) is list or type(output) is tuple:
        for i in range(len(output)):
            if not compare(output[i], truth[i], **kargs):
                print("TODO {} doesn't pass test: {}".format(todo, i))
                failed+=1
    else:
        if not compare(output, truth, **kargs):
            print("TODO {} doesn't pass test".format(todo))
            failed+=1
    return failed

HKD = features.HarrisKeypointDetector()
SFD = features.SimpleFeatureDescriptor()
MFD = features.MOPSFeatureDescriptor()
SSDFM = features.SSDFeatureMatcher()

image = np.array(Image.open('resources/triangle1.jpg'))
grayImage = cv2.cvtColor(image.astype(np.float32)/255.0, cv2.COLOR_BGR2GRAY)
def compute_and_save():
    (a,b) = HKD.computeHarrisValues(grayImage) # Todo1
    c = HKD.computeLocalMaxima(a) # Todo2
    d = HKD.detectKeypoints(image) # Todo3
    e = SFD.describeFeatures(image, d) # Todo 4
    f = MFD.describeFeatures(image, d) # Todo 5,6
    # No test for Todo 7 or 8
    d_proc = pickle_cv2(d)
    np.savez('resources/arrays',a=a,b=b,c=c,d_proc=d_proc,e=e,f=f)

try_this(1, HKD.computeHarrisValues, [loaded['a'],loaded['b']], compare_array, grayImage)
c,d=HKD.computeHarrisValues(grayImage)

# np.diagonal(c)[[0,20,33,48,123]]
# np.diagonal(d)[[0,20,33,48,123]]
# plt.figure(); plt.imshow(c, cmap = plt.get_cmap('gray'), vmax = 1)
# plt.figure(); plt.imshow(d, cmap = plt.get_cmap('gray'), vmax = 1)
ans_a=loaded['a']
ans_b=loaded['b']
# plt.figure(); plt.imshow(ans_a, cmap = plt.get_cmap('gray'), vmax = 1)
# plt.figure(); plt.imshow(ans_b, cmap = plt.get_cmap('gray'), vmax = 1)

# print(compare_array(c,ans_a))
# print(compare_array(d,ans_b))
# print(c,d)
# print([loaded['a'],loaded['b']])

# patch HKD so future tests won't fail because the last test failed
class HKD2(features.HarrisKeypointDetector):
    def computeHarrisValues(self,image):
        return loaded['a'],loaded['b']

    def computeLocalMaxima(harrisImage):
        '''
        Input:
            harrisImage -- numpy array containing the Harris score at
                           each pixel.
        Output:
            destImage -- numpy array containing True/False at
                         each pixel, depending on whether
                         the pixel value is the local maxima in
                         its 7x7 neighborhood.
        '''
        destImage = np.zeros_like(harrisImage, np.bool_)

            # TODO 2: Compute the local maxima image
    #         pad_num=3
    #         padded = np.pad(harrisImage, pad_num, 'minimum')
    #         for col in range(pad_num, pad_num+destImage.shape[0],1):
    #             for row in range(pad_num, pad_num+destImage.shape[1],1):
    #                 local_max=True
    #                 for i in range(-pad_num, pad_num):
    #                     for j in range(-pad_num, pad_num):
    #                         if padded[col][row]<padded[i+col][j+row]:
    #                             local_max=False
    #                 if local_max:
    #                     destImage[col-pad_num,row-pad_num]=True
        maxima=scipy.ndimage.maximum_filter(input=harrisImage, size=(7,7),mode='nearest')
        destImage=harrisImage==maxima
#         for col in range(0, destImage.shape[0],1):
#             for row in range(0, destImage.shape[1],1):
#                 if maxima[col,row]==0:
#                     destImage[col,row]=True
        return destImage
# HKD=HKD2()
# try_this(2, HKD.computeLocalMaxima, loaded['c'], compare_array, loaded['a'])
# ans_c=loaded['c']
# my_c=HKD2.computeLocalMaxima(loaded['a'])
# plt.figure(); plt.imshow(loaded['a'], cmap = plt.get_cmap('gray'), vmax = 1)
# plt.figure(); plt.imshow(ans_c, cmap = plt.get_cmap('gray'), vmax = 1)
# plt.figure(); plt.imshow(my_c, cmap = plt.get_cmap('gray'), vmax = 1)
# zeros = np.zeros_like(my_c)
# print(compare_array(my_c,zeros))
# class HKD3(HKD2):
#   def computeLocalMaxima(self,image):
#     return loaded['c']

# HKD=HKD3()

# try_this(3, HKD.detectKeypoints, d, compare_cv2_points, image)


Starting test for TODO 1
Starting test for TODO 3


AttributeError: 'numpy.ndarray' object has no attribute 'pt'